# Dependences

In [1]:
include("../../src/struct_data.jl")

FusionAggregate (generic function with 1 method)

# Force Functions

## Cubic

In [2]:
dump(Cubic)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: Cubic{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [3]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0
fₚ   = 0.1

0.1

In [4]:
Cubic(μ₁,rₘₐₓ,rₘᵢₙ)

Cubic{Float64}(1.0, 2.0, 3.0)

In [5]:
# Show function on specific r value
r = 1
force_func(Cubic(μ₁,rₘₐₓ,rₘᵢₙ),r)

4.0

In [6]:
dump(
        InteractionPar(
        Cubic(μ₁,rₘₐₓ,rₘᵢₙ),
        ContractilePar(fₚ)
    )
)

InteractionPar
  Force: Cubic{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.1


## LennardJones

In [7]:
dump(LennardJones)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: LennardJones{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


In [8]:
# Parameters
μ₁   = 1.0
rₘₐₓ = 2.0
rₘᵢₙ = 3.0

3.0

In [9]:
LennardJones(μ₁,rₘₐₓ,rₘᵢₙ)

LennardJones{Float64}(1.0, 2.0, 3.0)

In [10]:
# Show function on specific r value
r = 1.9
force_func(LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),r)

1.9609746568876094

In [11]:
dump(
    InteractionPar(
        LennardJones(μ₁,rₘₐₓ,rₘᵢₙ),
        ContractilePar(fₚ)
    )
)

InteractionPar
  Force: LennardJones{Float64}
    μ₁: Float64 1.0
    rₘᵢₙ: Float64 2.0
    rₘₐₓ: Float64 3.0
  Contractile: ContractilePar
    fₚ: Float64 0.1


# Model Parameter

In [12]:
Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Examples

In [13]:
X = Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
Y = Float64.(readdlm("../../data/init/Sphere/10.0.xyz")[3:end,2:end]) |> cu

746×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -2.89  -8.98
  0.5  -2.89  -8.98
  2.5  -2.89  -8.98
 -2.5  -1.15  -8.98
 -0.5  -1.15  -8.98
  1.5  -1.15  -8.98
  3.5  -1.15  -8.98
 -3.5   0.58  -8.98
 -1.5   0.58  -8.98
  0.5   0.58  -8.98
  2.5   0.58  -8.98
 -2.5   2.31  -8.98
 -0.5   2.31  -8.98
  ⋮           
  3.5  -2.31   8.98
 -3.5  -0.58   8.98
 -1.5  -0.58   8.98
  0.5  -0.58   8.98
  2.5  -0.58   8.98
 -2.5   1.15   8.98
 -0.5   1.15   8.98
  1.5   1.15   8.98
  3.5   1.15   8.98
 -1.5   2.89   8.98
  0.5   2.89   8.98
  2.5   2.89   8.98

## One Aggregate

In [14]:
@start_agg One = Aggregate(
    [AggType("HEK_1", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X)],
    [AggLocation("HEK_1",[0 0 0])],
    Model
)
show_aggregates(One)

========================= Type =======================


1-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×2504 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

Radius


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

------------------ Neighbors Size -------------------
idx      = (2504, 2504)
idx_red  = (21, 2504)
idx_sum  = (1, 2504)
idx_cont = (50, 2504)
------------------- Forces Size ---------------------
dX       = (2504, 3)
F        = (2504, 3)


## Simple Aggregate

In [15]:
@start_agg Simple = Aggregate(
    [AggType("HEK_1", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X)],
    [
        AggLocation("HEK_1",[100 0   0]),
        AggLocation("HEK_1",[0   100 0])
    ],
    Model
)
show_aggregates(Simple)

========================= Type =======================


1-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

Radius


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


## Complex Aggregate

In [16]:
@start_agg Complex = Aggregate(
    [
        AggType("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X),
        AggType("B", InteractionPar(Cubic(2.0,2.0,3.0), ContractilePar(0.02)),Y),
        AggType("C", InteractionPar(Cubic(3.0,2.0,3.0), ContractilePar(0.03)),X),
        AggType("D", InteractionPar(Cubic(4.0,2.0,3.0), ContractilePar(0.04)),Y),
    ],
    [
        AggLocation("A",[100 0   0  ]),
        AggLocation("C",[0   100 0  ]),
        AggLocation("D",[0   100 100]),
        AggLocation("B",[0   100 0  ])
    ],
    Model
)
show_aggregates(Complex)

========================= Type =======================


4-element Vector{AggType}:
 AggType("A", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("B", InteractionPar(Cubic{Float64}(2.0, 2.0, 3.0), ContractilePar(0.02)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("C", InteractionPar(Cubic{Float64}(3.0, 2.0, 3.0), ContractilePar(0.03)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("D", InteractionPar(Cubic{Float64}(4.0, 2.0, 3.0), ContractilePar(0.04)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×6500 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02])

Radius


4-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 10.43
 15.27
 10.43

------------------ Neighbors Size -------------------
idx      = (6500, 6500)
idx_red  = (21, 6500)
idx_sum  = (1, 6500)
idx_cont = (50, 6500)
------------------- Forces Size ---------------------
dX       = (6500, 3)
F        = (6500, 3)


## Fusion Aggregate

In [17]:
@start_agg FusionAGG = FusionAggregate(
    [AggType("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X)], 
    Model
) 
show_aggregates(FusionAGG)

========================= Type =======================


1-element Vector{AggType}:
 AggType("A", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

Radius


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


## Complex Fusion Aggregate

In [39]:
@time @start_agg FusionComplex = FusionAggregate(
    [
        AggType("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X),
        AggType("B", InteractionPar(Cubic(2.0,2.0,3.0), ContractilePar(0.02)),Y)
    ],
    Model
)
show_aggregates(FusionComplex)

# TEXT = hcat(Fusion_AGG_2.Outline,Matrix(Fusion_AGG_2.Position))
# open("tetero_3.xyz", "w") do f
#     write(f, "$(size(Fusion_AGG_2.Position, 1))\n")
#     write(f, "t=$(0)\n")
#     writedlm(f,TEXT, ' ')
# end


  0.165215 seconds (6.71 k allocations: 166.059 MiB, 11.11% gc time)
========================= Type =======================


2-element Vector{AggType}:
 AggType("A", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("B", InteractionPar(Cubic{Float64}(2.0, 2.0, 3.0), ContractilePar(0.02)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×3250 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02])

Radius


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 10.43

------------------ Neighbors Size -------------------
idx      = (3250, 3250)
idx_red  = (21, 3250)
idx_sum  = (1, 3250)
idx_cont = (50, 3250)
------------------- Forces Size ---------------------
dX       = (3250, 3)
F        = (3250, 3)
